In [134]:
import pandas as pd
import requests as req
import bs4
import xlsxwriter
import time

In [80]:
file_name = 'journal_links_db.xlsx'
xlsx = pd.ExcelFile(filename)
sheet_names = xlsx.sheet_names
# print(sheet_names)
# ['mk', 'ct', 'br', 'gu', 'er', 'bq', 'sq', 'pr', 'nm', 'ch']

['mk', 'ct', 'br', 'gu', 'er', 'bq', 'sq', 'pr', 'nm', 'ch']


https://doi.org/10.1148/radiol.211818


In [133]:
res = req.get(df.iloc[0].values[0])
soup = bs4.BeautifulSoup(res.content, 'html.parser')
print(soup)


<!DOCTYPE html>

<html class="pb-page" data-request-id="509a5c2b-9910-4ca8-ab0a-a5293151b1d7" lang="en"><head data-pb-dropzone="head"><meta content=";subPage:string:Status ;website:website:rsna-site;wgroup:string:RSNA Publication Websites;pageGroup:string:Error;page:string:IP blocked" name="pbContext"/>
<meta charset="utf-8"/>
<meta content="noarchive,noindex,nofollow" name="robots">
<meta content="Website" property="og:type"/>
<meta content="RSNA Publications Online" property="og:site_name"/>
<meta content="width=device-width,initial-scale=1" name="viewport"/>
<title>RSNA Publications Online | Error</title>
<link href="/wro/lprv~article-metrics-phase2.css" rel="stylesheet" type="text/css"/>
<link href="/wro/lprv~product.css" rel="stylesheet" type="text/css"/>
<link href="/products/rsna/releasedAssets/css/build-cf761ac921d1a80a232f.css" rel="stylesheet"><link href="/products/rsna/releasedAssets/css/print-cf761ac921d1a80a232f.css" media="print" rel="stylesheet">
<link data-pb-css-id="t

In [125]:
def get_abstract(soup):
    try:
        abstract_header = soup.find(class_='sectionInfo abstractSectionHeading')
        print("abstract" + abstract_header.get_text())
        return abstract_header.get_text()
    except:
        print('err')
        return 'err'

def get_authors(soup):
    try:
        names = ""
        authors_class = soup.find(class_='rlist--inline loa hidden-xs loa-accordion')
        authors = authors_class.findChildren("a")
        for author in authors:
            name = author.get_text()
            if name:
                #add names to list and remove whitespace
                names += (name.strip()) + ","
        print(names)
        return names[:-1] # remove last comma 
    except:
        print('err')
        return 'err'

def get_title(soup):
    try:
        title = soup.find(class_='citation__title')
        print(title)
        return title.get_text()
    except:
        print('err')
        return 'err'

In [126]:
# create workbook for the database
workbook = xlsxwriter.Workbook("database.xlsx")

In [127]:
# script to write all journal info to database
# different categories will be written on different sheets named 'mk', 'ct', ...
for category in sheet_names:
    print("CATEGORY############################################" + category)
    # some pandas-excel-xlsx magic here... ¯\_(ツ)_/¯
    df = pd.read_excel(file_name, sheet_name=category)
    worksheet = workbook.add_worksheet(category)
    worksheet.write(0, 0, 'title')
    worksheet.write(0, 1, 'authors')
    worksheet.write(0, 2, 'abstract')
    for i in range(0, 3): # reading links from pandas_tables starts at index 0
        link = df.iloc[i].values[0]
        print(str(i) + " " + link)
        # get article
        response = req.get(link)
        article_soup = bs4.BeautifulSoup(response.content, 'html.parser')
        # we have to increment i by 0 so we can have headers in index 0
        worksheet.write(i + 1, 0, get_title(article_soup))
        worksheet.write(i + 1, 1, get_authors(article_soup))
        worksheet.write(i + 1, 2, get_abstract(article_soup))
        time.wait(30)
    workbook.close()

CATEGORY############################################mk
0 https://doi.org/10.1148/radiol.211818

<!DOCTYPE html>

<html class="pb-page" data-request-id="65f16b37-8f07-40c2-a887-8c7dffa61570" lang="en"><head data-pb-dropzone="head"><meta content=";subPage:string:Status ;website:website:rsna-site;wgroup:string:RSNA Publication Websites;pageGroup:string:Error;page:string:IP blocked" name="pbContext"/>
<meta charset="utf-8"/>
<meta content="noarchive,noindex,nofollow" name="robots">
<meta content="Website" property="og:type"/>
<meta content="RSNA Publications Online" property="og:site_name"/>
<meta content="width=device-width,initial-scale=1" name="viewport"/>
<title>RSNA Publications Online | Error</title>
<link href="/wro/lprv~article-metrics-phase2.css" rel="stylesheet" type="text/css"/>
<link href="/wro/lprv~product.css" rel="stylesheet" type="text/css"/>
<link href="/products/rsna/releasedAssets/css/build-cf761ac921d1a80a232f.css" rel="stylesheet"><link href="/products/rsna/releasedAss


<!DOCTYPE html>

<html class="pb-page" data-request-id="dab13329-acf5-4dd2-becd-8b2a59eeb7fe" lang="en"><head data-pb-dropzone="head"><meta content=";subPage:string:Status ;website:website:rsna-site;wgroup:string:RSNA Publication Websites;pageGroup:string:Error;page:string:IP blocked" name="pbContext"/>
<meta charset="utf-8"/>
<meta content="noarchive,noindex,nofollow" name="robots">
<meta content="Website" property="og:type"/>
<meta content="RSNA Publications Online" property="og:site_name"/>
<meta content="width=device-width,initial-scale=1" name="viewport"/>
<title>RSNA Publications Online | Error</title>
<link href="/wro/lprv~article-metrics-phase2.css" rel="stylesheet" type="text/css"/>
<link href="/wro/lprv~product.css" rel="stylesheet" type="text/css"/>
<link href="/products/rsna/releasedAssets/css/build-cf761ac921d1a80a232f.css" rel="stylesheet"><link href="/products/rsna/releasedAssets/css/print-cf761ac921d1a80a232f.css" media="print" rel="stylesheet">
<link data-pb-css-id="t


<!DOCTYPE html>

<html class="pb-page" data-request-id="fc2e3626-f1bb-46cf-a6be-9231b79f033f" lang="en"><head data-pb-dropzone="head"><meta content=";subPage:string:Status ;website:website:rsna-site;wgroup:string:RSNA Publication Websites;pageGroup:string:Error;page:string:IP blocked" name="pbContext"/>
<meta charset="utf-8"/>
<meta content="noarchive,noindex,nofollow" name="robots">
<meta content="Website" property="og:type"/>
<meta content="RSNA Publications Online" property="og:site_name"/>
<meta content="width=device-width,initial-scale=1" name="viewport"/>
<title>RSNA Publications Online | Error</title>
<link href="/wro/lprv~article-metrics-phase2.css" rel="stylesheet" type="text/css"/>
<link href="/wro/lprv~product.css" rel="stylesheet" type="text/css"/>
<link href="/products/rsna/releasedAssets/css/build-cf761ac921d1a80a232f.css" rel="stylesheet"><link href="/products/rsna/releasedAssets/css/print-cf761ac921d1a80a232f.css" media="print" rel="stylesheet">
<link data-pb-css-id="t


<!DOCTYPE html>

<html class="pb-page" data-request-id="5bb7074f-48a1-4e69-97cf-25720239a48d" lang="en"><head data-pb-dropzone="head"><meta content=";subPage:string:Status ;website:website:rsna-site;wgroup:string:RSNA Publication Websites;pageGroup:string:Error;page:string:IP blocked" name="pbContext"/>
<meta charset="utf-8"/>
<meta content="noarchive,noindex,nofollow" name="robots">
<meta content="Website" property="og:type"/>
<meta content="RSNA Publications Online" property="og:site_name"/>
<meta content="width=device-width,initial-scale=1" name="viewport"/>
<title>RSNA Publications Online | Error</title>
<link href="/wro/lprv~article-metrics-phase2.css" rel="stylesheet" type="text/css"/>
<link href="/wro/lprv~product.css" rel="stylesheet" type="text/css"/>
<link href="/products/rsna/releasedAssets/css/build-cf761ac921d1a80a232f.css" rel="stylesheet"><link href="/products/rsna/releasedAssets/css/print-cf761ac921d1a80a232f.css" media="print" rel="stylesheet">
<link data-pb-css-id="t


<!DOCTYPE html>

<html class="pb-page" data-request-id="6807e277-27ee-4afc-af0d-7989de86a954" lang="en"><head data-pb-dropzone="head"><meta content=";subPage:string:Status ;website:website:rsna-site;wgroup:string:RSNA Publication Websites;pageGroup:string:Error;page:string:IP blocked" name="pbContext"/>
<meta charset="utf-8"/>
<meta content="noarchive,noindex,nofollow" name="robots">
<meta content="Website" property="og:type"/>
<meta content="RSNA Publications Online" property="og:site_name"/>
<meta content="width=device-width,initial-scale=1" name="viewport"/>
<title>RSNA Publications Online | Error</title>
<link href="/wro/lprv~article-metrics-phase2.css" rel="stylesheet" type="text/css"/>
<link href="/wro/lprv~product.css" rel="stylesheet" type="text/css"/>
<link href="/products/rsna/releasedAssets/css/build-cf761ac921d1a80a232f.css" rel="stylesheet"><link href="/products/rsna/releasedAssets/css/print-cf761ac921d1a80a232f.css" media="print" rel="stylesheet">
<link data-pb-css-id="t

...


<!DOCTYPE html>

<html class="pb-page" data-request-id="234f124e-6e3a-422f-a3c3-cb03590edab6" lang="en"><head data-pb-dropzone="head"><meta content=";subPage:string:Status ;website:website:rsna-site;wgroup:string:RSNA Publication Websites;pageGroup:string:Error;page:string:IP blocked" name="pbContext"/>
<meta charset="utf-8"/>
<meta content="noarchive,noindex,nofollow" name="robots">
<meta content="Website" property="og:type"/>
<meta content="RSNA Publications Online" property="og:site_name"/>
<meta content="width=device-width,initial-scale=1" name="viewport"/>
<title>RSNA Publications Online | Error</title>
<link href="/wro/lprv~article-metrics-phase2.css" rel="stylesheet" type="text/css"/>
<link href="/wro/lprv~product.css" rel="stylesheet" type="text/css"/>
<link href="/products/rsna/releasedAssets/css/build-cf761ac921d1a80a232f.css" rel="stylesheet"><link href="/products/rsna/releasedAssets/css/print-cf761ac921d1a80a232f.css" media="print" rel="stylesheet">
<link data-pb-css-id="t


<!DOCTYPE html>

<html class="pb-page" data-request-id="0b8106ba-62a4-4542-ab05-b52f1e42eeb2" lang="en"><head data-pb-dropzone="head"><meta content=";subPage:string:Status ;website:website:rsna-site;wgroup:string:RSNA Publication Websites;pageGroup:string:Error;page:string:IP blocked" name="pbContext"/>
<meta charset="utf-8"/>
<meta content="noarchive,noindex,nofollow" name="robots">
<meta content="Website" property="og:type"/>
<meta content="RSNA Publications Online" property="og:site_name"/>
<meta content="width=device-width,initial-scale=1" name="viewport"/>
<title>RSNA Publications Online | Error</title>
<link href="/wro/lprv~article-metrics-phase2.css" rel="stylesheet" type="text/css"/>
<link href="/wro/lprv~product.css" rel="stylesheet" type="text/css"/>
<link href="/products/rsna/releasedAssets/css/build-cf761ac921d1a80a232f.css" rel="stylesheet"><link href="/products/rsna/releasedAssets/css/print-cf761ac921d1a80a232f.css" media="print" rel="stylesheet">
<link data-pb-css-id="t


<!DOCTYPE html>

<html class="pb-page" data-request-id="7eaff1fc-25f5-4cbe-a27c-379608d33203" lang="en"><head data-pb-dropzone="head"><meta content=";subPage:string:Status ;website:website:rsna-site;wgroup:string:RSNA Publication Websites;pageGroup:string:Error;page:string:IP blocked" name="pbContext"/>
<meta charset="utf-8"/>
<meta content="noarchive,noindex,nofollow" name="robots">
<meta content="Website" property="og:type"/>
<meta content="RSNA Publications Online" property="og:site_name"/>
<meta content="width=device-width,initial-scale=1" name="viewport"/>
<title>RSNA Publications Online | Error</title>
<link href="/wro/lprv~article-metrics-phase2.css" rel="stylesheet" type="text/css"/>
<link href="/wro/lprv~product.css" rel="stylesheet" type="text/css"/>
<link href="/products/rsna/releasedAssets/css/build-cf761ac921d1a80a232f.css" rel="stylesheet"><link href="/products/rsna/releasedAssets/css/print-cf761ac921d1a80a232f.css" media="print" rel="stylesheet">
<link data-pb-css-id="t


<!DOCTYPE html>

<html class="pb-page" data-request-id="5207e292-44f3-4618-9533-d078700749b0" lang="en"><head data-pb-dropzone="head"><meta content=";subPage:string:Status ;website:website:rsna-site;wgroup:string:RSNA Publication Websites;pageGroup:string:Error;page:string:IP blocked" name="pbContext"/>
<meta charset="utf-8"/>
<meta content="noarchive,noindex,nofollow" name="robots">
<meta content="Website" property="og:type"/>
<meta content="RSNA Publications Online" property="og:site_name"/>
<meta content="width=device-width,initial-scale=1" name="viewport"/>
<title>RSNA Publications Online | Error</title>
<link href="/wro/lprv~article-metrics-phase2.css" rel="stylesheet" type="text/css"/>
<link href="/wro/lprv~product.css" rel="stylesheet" type="text/css"/>
<link href="/products/rsna/releasedAssets/css/build-cf761ac921d1a80a232f.css" rel="stylesheet"><link href="/products/rsna/releasedAssets/css/print-cf761ac921d1a80a232f.css" media="print" rel="stylesheet">
<link data-pb-css-id="t

KeyboardInterrupt: 

'Giovanni Foti,Chiara Longo,Mirko D’Onofrio,Simone Natali,Gianluca Piovan,Eugenio Oliboni,Venanzio Iacono,Massimo Guerriero,Claudio Zorzi'

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [130]:
get_title(soup)

None
err


'err'